# Companies House cultural data

Extracting data for cultural organisations from Companies House data based on SIC code.

## Setup environment

In [14]:
import json

import geopandas as gpd
import pandas as pd
import pipeline_utils.db as db
from pipeline_utils.filesystem.paths import DATA, RAW_DATA, REF_DATA, SITE

In [15]:
OUT = SITE / 'data/companies-house/_data/'
OUT.mkdir(exist_ok=True, parents=True)

## Extract Companies House data

Using a  shortlist of SIC codes, extract data from Companies House.

In [16]:
db = db.connect(read_only=True)

Create temporary tables

In [17]:
db.query('''
    CREATE TEMP TABLE tSicCodes AS SELECT * FROM read_csv('../raw/sic_codes.csv');
''')

In [18]:
db.query('''
    CREATE TEMP TABLE tPostcodes AS
        SELECT pcds AS postcode, lat, long
        FROM read_csv('../data/reference/onspd_extract.csv')
        WHERE oslaua == 'E08000021';
''')

In [19]:
db.query('''
    CREATE TEMP TABLE tCompanies AS SELECT 
        CompanyName as registered_name,
        CompanyNumber as company_number,
        "RegAddress.PostCode" as postcode,
        [x for x in [
            "SICCode.SicText_1",
            "SICCode.SicText_2",
            "SICCode.SicText_3",
            "SICCode.SicText_4"
        ] if x is not NULL] as sic_code,
        lat, long
    FROM CompanyData c
    JOIN tPostcodes p
    ON c."RegAddress.PostCode" == p.postcode;
''')

Query the shortlist codes

In [20]:
shortlist_sic_codes = db.query('''SELECT * FROM tSicCodes;''').df()

Query the companies data

In [21]:
culture_companies = db.query('''
    SELECT DISTINCT c.*
        FROM tCompanies c
        JOIN tSicCodes s
        ON list_contains(c.sic_code, s.sic_code)
        ORDER BY c.company_number;
''').df()

In [22]:
db.close()

In [23]:
culture_companies.sic_code = culture_companies.sic_code.map(lambda x: x.tolist())

In [24]:
culture_companies

,registered_name,company_number,postcode,sic_code,lat,long
0,PEOPLE'S THEATRE ARTS GROUP LIMITED,00242886,NE6 5QF,"[90010 - Performing arts, 90030 - Artistic cre...",54.990445,-1.584953
1,INSTITUTE OF AMATEUR CINEMATOGRAPHERS LIMITED(...,00269085,NE3 2DT,[63990 - Other information service activities ...,55.023848,-1.620913
2,NEWCASTLE PEOPLE'S THEATRE ARTS TRUST LIMITED,00393739,NE6 5QF,[90040 - Operation of arts facilities],54.990445,-1.584953
3,MAWSON & WAREHAM (MUSIC) LIMITED,00957980,NE1 5BP,"[58190 - Other publishing activities, 59112 - ...",54.973132,-1.613142
4,TYNESIDE CINEMA,01113101,NE1 6QG,[59140 - Motion picture projection activities],54.973801,-1.611761
...,...,...,...,...,...,...
502,HARVEY DUCKMAN LTD,15879932,NE1 1JF,[90030 - Artistic creation],54.969968,-1.613654
503,BUNNY RWLK LTD,15881018,NE1 5UD,"[90010 - Performing arts, 90030 - Artistic cre...",54.971137,-1.618832
504,AVERIA AGENCY UK LTD,15888622,NE3 1YQ,"[63120 - Web portals, 90030 - Artistic creation]",55.002906,-1.608369
505,GEORDIE VISION LTD,15906324,NE6 2HL,[59112 - Video production activities],54.970149,-1.581302


## Save the companies data

In [25]:
culture_companies.sort_values('company_number').to_csv(OUT / 'list.csv', index=False)

In [26]:
geo = gpd.GeoDataFrame(
    culture_companies,
    geometry=gpd.points_from_xy(culture_companies.long, culture_companies.lat, crs='epsg:4326')
)

geo.set_crs(None, allow_override=True).to_file(OUT / 'company_locations.geojson')

/home/giles/.local/share/virtualenvs/northumbria-culture-connect-dQ7VTDtt/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


## Process SIC codes

In [27]:
sic_lookup = pd.read_csv(REF_DATA / 'sic-lookup.csv', dtype=str).set_index('code')

In [28]:
all_sic_codes = (
    culture_companies
        .explode('sic_code')
        .groupby('sic_code')
        .company_number.count()
        .sort_values(ascending=False)
        .reset_index()
        .rename(columns={ 'company_number': 'count'})
)

all_sic_codes[all_sic_codes.sic_code.isin(shortlist_sic_codes.sic_code)].to_csv(OUT / 'sic_codes.csv')

all_sic_codes['id'] = all_sic_codes.sic_code.str.extract(r'^(\d+)')

all_sic_codes = (
    all_sic_codes
        .merge(sic_lookup, left_on='id', right_index=True)
        .drop(['sic_code'], axis=1)
        .rename(columns={'section': 'group'})
)

## Calculate linked SIC codes

In [29]:
nodes = all_sic_codes.to_dict(orient='records')

In [30]:
sources = [n['id'] for n in nodes]

In [31]:
network = culture_companies.loc[:, ['sic_code', 'company_number']]
network.sic_code = network.sic_code.apply(lambda v: [x.split(' ')[0] for x in v])

In [32]:
def count_others(v):
    edges = (
        network[network.sic_code.apply(lambda x: v in x)]
        .explode('sic_code')
        .groupby('sic_code')
        .company_number.count()
        .reset_index()
        .rename(columns={'company_number': 'weight', 'sic_code': 'target'})\
    )
    edges['source'] = v

    return edges.loc[(edges.source != edges.target), ['source', 'target', 'weight']].sort_values('weight', ascending=False)

directed_pairs = pd.concat([count_others(s) for s in sources]).reset_index(drop=True)

Edges will contain 

In [33]:
pairs = pd.DataFrame(
    directed_pairs[['source', 'target']]
        .stack()
        .reset_index(level=1, drop=True)
        .pipe(lambda e: e.groupby(e.index).apply(lambda x: sorted(list(x))))
        .pipe(lambda e: e[e.duplicated()])
        .reset_index(drop=True)
        .tolist(),
    columns=['source', 'target']
)

edges = pairs.merge(directed_pairs).sort_values('weight', ascending=False).to_dict(orient='records')

In [34]:
pd.DataFrame(edges)

,source,target,weight
0,90010,90030,15
1,85520,85600,13
2,90010,90020,12
3,85520,85590,12
4,59111,59112,11
...,...,...,...
343,74990,94990,1
344,59112,74901,1
345,74901,79120,1
346,59200,86220,1


In [35]:
with open(OUT / 'graph.json', 'w') as graph:
    json.dump({ 'nodes': nodes, 'edges': edges }, fp=graph, indent=2)